In [1]:
#importing relevant libraries

import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn.functional as F
import os
import torch.quantization
import copy

%cd ..

from CNN_small_architecture import CNNSmall
from Tests.helper_functions import tokenize

/home/emil/Documents/bachelor/bachelorprojekt/Python


In [2]:
MNIST_test = datasets.MNIST(root='./data', train=False, download=True, transform=torchvision.transforms.ToTensor())
test_set = [[data[0], tokenize(data[1])] for data in MNIST_test if data[1] in [1,2]][:1000]

batch_size = 1
test_loader = DataLoader(test_set, batch_size=batch_size)

In [17]:
model_original = CNNSmall()

path = "CNN_small"
load = True

if load and os.path.isfile(path):
    model_original.load_state_dict(torch.load(path))

quantized_model = torch.quantization.quantize_dynamic(
    copy.deepcopy(model_original), qconfig_spec={nn.Conv2d, nn.BatchNorm2d, nn.Linear}, dtype=torch.qint8
)

quantized_model.eval()

preds = []

In [18]:
quantized_model

CNNSmall(
  (conv1): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1))
  (batchNorm1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (maxPool1): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(3, 5, kernel_size=(5, 5), stride=(1, 1))
  (batchNorm2): BatchNorm2d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (maxPool2): MaxPool2d(kernel_size=(3, 3), stride=(3, 3), padding=0, dilation=1, ceil_mode=False)
  (lin): DynamicQuantizedLinear(in_features=45, out_features=2, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
  (softmax): Softmax(dim=1)
  (network): Sequential(
    (0): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(3, 5, ke

In [10]:
with torch.no_grad():
    for data in test_loader:
        image, label = data
        output = quantized_model(image)
        if torch.argmax(output) == torch.argmax(label):
            preds.append(1)
        else:
            preds.append(0)

In [11]:
preds.count(1)/len(preds)

0.994